In [ ]:
# =========================
# 1. Imports
# =========================
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils import class_weight

# =========================
# 2. Data Directories & Params
# =========================
train_dir = '../data/train'
val_dir = '../data/val'
test_dir = '../data/test'
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# =========================
# 3. Data Generators
# =========================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='binary', shuffle=True
)
validation_generator = val_datagen.flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='binary', shuffle=False
)
test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='binary', shuffle=False
)

# =========================
# 4. Class Weights
# =========================
cw = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(cw))
print("Computed Class Weights:", class_weights)

# =========================
# 5. Autoencoder Definition
# =========================
input_img = Input(shape=(224, 224, 3))
# Encoder
x = Conv2D(32, (3,3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2,2), padding='same')(x)
# Decoder
x = Conv2D(64, (3,3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = UpSampling2D((2,2))(x)
decoded = Conv2D(3, (3,3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
encoder = Model(input_img, encoded)
autoencoder.compile(optimizer='adam', loss='mse')

# =========================
# 6. Autoencoder Training
# =========================
autoencoder_path = "autoencoder_model.h5"
encoder_path = "encoder_model.h5"

if os.path.exists(autoencoder_path) and os.path.exists(encoder_path):
    print("Loading pre-trained Autoencoder and Encoder...")
    autoencoder = load_model(autoencoder_path)
    encoder = load_model(encoder_path)
else:
    print("Training Autoencoder...")
    # Train autoencoder with image generators, only images as input
    ae_train_generator = train_datagen.flow_from_directory(
        train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode=None, shuffle=True)
    ae_val_generator = val_datagen.flow_from_directory(
        val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode=None, shuffle=False)

    # Custom generator for autoencoder fitting
    def autoencoder_gen(gen):
        for batch in gen:            yield (batch, batch)
    steps_per_epoch = len(ae_train_generator)
    val_steps = len(ae_val_generator)
    autoencoder.fit(
        autoencoder_gen(ae_train_generator), 
        steps_per_epoch=steps_per_epoch,
        epochs=10,
        validation_data=autoencoder_gen(ae_val_generator),
        validation_steps=val_steps
    )
    autoencoder.save(autoencoder_path)
    encoder.save(encoder_path)
    print("Autoencoder & Encoder Saved!")

# =========================
# 7. Reconstruct Datasets Using Autoencoder
# =========================
def build_reconstructed_dataset(labeled_gen, model_ae):
    x_list, y_list = [], []
    labeled_gen.reset()
    for _ in range(len(labeled_gen)):
        x_batch, y_batch = next(labeled_gen)
        x_recon = model_ae.predict(x_batch, verbose=0)
        x_list.append(x_recon)
        y_list.append(y_batch)
    return np.concatenate(x_list), np.concatenate(y_list)
print("Reconstructing datasets...")
X_train_recon, y_train = build_reconstructed_dataset(train_generator, autoencoder)
X_val_recon, y_val = build_reconstructed_dataset(validation_generator, autoencoder)
X_test_recon, y_test = build_reconstructed_dataset(test_generator, autoencoder)
print("Train:", X_train_recon.shape, "Val:", X_val_recon.shape, "Test:", X_test_recon.shape)

# =========================
# 8. Classifier on Top of Encoder
# =========================
encoder.trainable = False # freeze encoder

x = Flatten()(encoder.output) # crucial: flatten output for Dense layers
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
classifier_output = Dense(1, activation='sigmoid')(x)
classifier = Model(encoder.input, classifier_output, name='classifier')
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# =========================
# 9. Train the Classifier
# =========================
print("Training classifier...")
history = classifier.fit(
    ''
    X_train_recon, y_train,
    batch_size=BATCH_SIZE,
    epochs=10,
    validation_data=(X_val_recon, y_val),
    class_weight=class_weights,
    callbacks=[lr_scheduler, early_stop]
)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Computed Class Weights: {0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}
Training Autoencoder...
Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 348s 2s/step - loss: 0.0069 - val_loss: 0.0018
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 312s 2s/step - loss: 0.0011 - val_loss: 0.0016
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 343s 2s/step - loss: 8.2279e-04 - val_loss: 0.0013
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 262s 2s/step - loss: 6.4812e-04 - val_loss: 0.0013
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 325s 2s/step - loss: 5.9730e-04 - val_loss: 0.0012
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 300s 2s/step - loss: 5.4644e-04 - val_loss: 0.0011
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 301s 2s/step - loss: 4.8559e-04 - val_loss: 0.0010
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 247s 2s

Autoencoder & Encoder Saved!
Reconstructing datasets...
Train: (5216, 224, 224, 3) Val: (16, 224, 224, 3) Test: (624, 224, 224, 3)
Training classifier...
Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 141s 745ms/step - accuracy: 0.7184 - loss: 1.0394 - val_accuracy: 0.8750 - val_loss: 0.4983 - learning_rate: 0.0010
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 121s 744ms/step - accuracy: 0.7893 - loss: 0.4208 - val_accuracy: 0.6250 - val_loss: 0.5432 - learning_rate: 0.0010
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 119s 731ms/step - accuracy: 0.7345 - loss: 0.4260 - val_accuracy: 0.8750 - val_loss: 0.4334 - learning_rate: 0.0010
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 119s 729ms/step - accuracy: 0.7044 - loss: 0.4409 - val_accuracy: 0.8125 - val_loss: 0.4566 - learning_rate: 0.0010
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 118s 725ms/step - accuracy: 0.6716 - loss: 0.4338 - val_accuracy: 0.9375 - val_loss: 0.4220 - learning_rate: 0.0010
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 118s 724ms/step - accuracy

In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load ResNet50 base without top layers, freeze initially
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
resnet_base.trainable = False

# Build ResNet50 model
resnet_input = Input(shape=(224,224,3))
x = resnet_base(resnet_input, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
resnet_output = Dense(1, activation='sigmoid')(x)
resnet_model = Model(resnet_input, resnet_output)

# Compile
resnet_model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Train on reconstructed images
print("Training ResNet50 on autoencoder-reconstructed images...")
history_resnet = resnet_model.fit(
    X_train_recon, y_train,
    batch_size=BATCH_SIZE,
    epochs=10,
    validation_data=(X_val_recon, y_val),
    class_weight=class_weights,
    callbacks=[lr_scheduler, early_stop]
)

# Optional: fine-tune whole ResNet50 with lower LR
resnet_base.trainable = True
resnet_model.compile(optimizer=Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])
print("Fine-tuning ResNet50...")
history_resnet_ft = resnet_model.fit(
    X_train_recon, y_train,
    batch_size=BATCH_SIZE,
    epochs=5,
    validation_data=(X_val_recon, y_val),
    class_weight=class_weights,
    callbacks=[lr_scheduler, early_stop]
)

# Evaluate on reconstructed test set
test_loss, test_acc = resnet_model.evaluate(X_test_recon, y_test, batch_size=BATCH_SIZE)
print(f"ResNet50 Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


Training ResNet50 on autoencoder-reconstructed images...
Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 274s 2s/step - accuracy: 0.4965 - loss: 0.7571 - val_accuracy: 0.5625 - val_loss: 0.6779 - learning_rate: 1.0000e-04
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 277s 2s/step - accuracy: 0.5656 - loss: 0.6771 - val_accuracy: 0.5625 - val_loss: 0.6623 - learning_rate: 1.0000e-04
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5838 - loss: 0.6623
Epoch 3: ReduceLROnPlateau reducing learning rate to 2.9999999242136255e-05.
163/163 ━━━━━━━━━━━━━━━━━━━━ 275s 2s/step - accuracy: 0.6192 - loss: 0.6472 - val_accuracy: 0.5625 - val_loss: 0.6775 - learning_rate: 1.0000e-04
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 276s 2s/step - accuracy: 0.6823 - loss: 0.6336 - val_accuracy: 0.6250 - val_loss: 0.6430 - learning_rate: 3.0000e-05
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 278s 2s/step - accuracy: 0.7147 - loss: 0.6234 - val_accuracy: 0.8125 - val_loss: 0.6399 - learning_rate: 3.0000e-05
Fine-tu